In [104]:
from query_graph import QueryGraph
import pandas as pd
from tqdm import tqdm
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import FOAF, RDF
import json

In [68]:
query = [["?x", "2", "2"],
        ["?x", "2", "3"]]

In [73]:
g = QueryGraph(3, 2, 3, 1)

In [74]:
g.cardinality = 4
for q in query:
    g.add_triple(q)

In [75]:
g

In [76]:
g.create_graph()

(array([[0, 1, 0],
        [0, 1, 1],
        [0, 0, 0]], dtype=uint8),
 array([[1, 0]], dtype=uint8),
 array([[[0, 0, 0],
         [0, 0, 0],
         [1, 1, 0]]], dtype=uint8),
 4)

In [58]:
g.print()

Nodes {'?x', '3', '2'}, Edges {'1'}, Triples [['?x', '1', '2'], ['?x', '1', '3']]


## Transform kg triple file


In [59]:
file = open("/home/tim/cardinality_estimator/Freebase/train.txt")
lines = file.readlines()

In [83]:
mapping = {}
df = pd.DataFrame(columns=["s","p","o"])
node_counter = 0
predicate_counter = 0
for line in tqdm(lines):
    els = line.split()
    # Map the terms to ints, if not happened already
    if not els[0] in mapping:
        mapping[els[0]] = node_counter
        node_counter += 1
    if not els[1] in mapping:
        mapping[els[1]] = predicate_counter
        predicate_counter += 1
    if not els[2] in mapping:
        mapping[els[2]] = node_counter
        node_counter += 1
    df = df.append({"s": mapping[els[0]], "p": mapping[els[1]], "o": mapping[els[2]]}, ignore_index=True)
        
    


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 272115/272115 [13:27<00:00, 336.99it/s]


In [108]:
df

,s,p,o
0,0,0,1
1,2,1,3
2,4,2,5
3,6,3,7
4,8,4,9
...,...,...,...
272110,7701,11,23
272111,2483,20,2423
272112,6954,170,8974
272113,3818,30,7570


In [111]:
df["p"].max()

236

In [102]:
g = Graph()

for idx, row in tqdm(df.iterrows()):
    e1 = URIRef("http://example.org/" + str(row[0]))
    e2 = URIRef("http://example.org/" + str(row[1]))
    e3 = URIRef("http://example.org/" + str(row[2]))
    g.add((e1, e2, e3))


272115it [00:12, 22120.01it/s]


In [103]:
print(g.serialize(format="nt", destination="freebase_int.ttl"))

/home/tim/anaconda3/envs/lmkg-s/lib/python3.7/site-packages/rdflib/plugins/serializers/nt.py:36: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  "NTSerializer always uses UTF-8 encoding. "


[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


### Custom reader

In [105]:
with open("final_datasets/freebase_train_test_data.json") as f:
     data = json.load(f)

In [127]:
for query in data:
    print(query["triples"])
    print(query["y"])
    g = QueryGraph(14505, 237, 3, 2)
    for tp in query["triples"]:
        g.add_triple([el.replace("<http://example.org/", "").replace(">","") for el in tp if el is not "."])
    g.cardinality = query["y"]
    x, ep, a, cardinality = g.create_graph()


[['<http://example.org/13443>', '<http://example.org/4>', '?o1', '.'], ['<http://example.org/13443>', '?p1', '<http://example.org/117>', '.']]
8
[['<http://example.org/1535>', '?p1', '?o1', '.'], ['<http://example.org/1535>', '?p2', '<http://example.org/746>', '.']]
78
[['?s', '?p1', '<http://example.org/3057>', '.'], ['?s', '?p2', '<http://example.org/2883>', '.']]
1
[['<http://example.org/2522>', '?p1', '<http://example.org/8164>', '.'], ['<http://example.org/2522>', '<http://example.org/66>', '?o1', '.']]
1
[['?s', '<http://example.org/26>', '<http://example.org/342>', '.'], ['?s', '?p1', '<http://example.org/7246>', '.']]
4
[['<http://example.org/1052>', '<http://example.org/17>', '?o1', '.'], ['<http://example.org/1052>', '<http://example.org/48>', '?o2', '.']]
2
[['<http://example.org/1174>', '<http://example.org/19>', '?o1', '.'], ['<http://example.org/1174>', '<http://example.org/212>', '?o2', '.']]
10
[['<http://example.org/19>', '<http://example.org/9>', '?o1', '.'], ['<http:

In [123]:
x

array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1],
       [0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [124]:
ep

array([[0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [125]:
a

array([[[0, 0, 0],
        [0, 0, 1],
        [0, 0, 0]],

       [[0, 0, 0],
        [1, 0, 0],
        [0, 0, 0]]], dtype=uint8)